In [1]:
import pandas as pd
import numpy as np
import nltk
import pickle
from nltk import word_tokenize, RegexpTokenizer,PunktSentenceTokenizer, sent_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
#nltk.download('stopwords')
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [2]:
def prepro(query):
    stop_words = stopwords.words('english')
    lemmatiser = WordNetLemmatizer()
    word_stemmer = PorterStemmer()
    count = 0
    tokens = gensim.utils.simple_preprocess(str(query), deacc=True)
    count += len(tokens)
    cleaned = [word for word in tokens if word not in stop_words]
    lemmatized = [lemmatiser.lemmatize(word_stemmer.stem(word)) for word in cleaned]
    return lemmatized

In [3]:
def normalise(query):
    tokens = gensim.utils.simple_preprocess(str(query), deacc=True)
    return tokens

In [29]:
prepro('machine learning')

['machin', 'learn']

In [4]:
all_info = pd.read_csv('graph_data_final.csv')

In [20]:
all_info.columns

Index(['id', 'author', 'title', 'publication_date', 'publication_type',
       'abstract', 'language', 'journal', 'organisation_unit_code',
       'organisation', 'department'],
      dtype='object')

In [21]:
all_info.head()

,id,author,title,publication_date,publication_type,abstract,language,journal,organisation_unit_code,organisation,department
0,108069.0,"Schützeichel, Rainer",Die Perspektive des Städtebauers. Ein Blick au...,2016.0,Book Chapter,NaN,de,Grundlagen,NaN,NaN,NaN
1,127413.0,"Schützeichel, Rainer",Einleitung. Wegmarken einer Theorie des archit...,2016.0,Book Chapter,NaN,de,NaN,NaN,NaN,NaN
2,127410.0,"Schützeichel, Rainer",Tradition as a Means of Modernisation: The Cru...,2016.0,Book Chapter,NaN,en,NaN,NaN,NaN,NaN
3,28599.0,"Schützeichel, Rainer",Architettura contestuale. Difesa di una proget...,2010.0,Book Chapter,NaN,it,NaN,NaN,NaN,NaN
4,319574.0,"Schützeichel, Rainer",Die Stadt und das Haus,2010.0,Book Chapter,NaN,en||de,NaN,NaN,NaN,NaN


In [82]:
def create_list(query_type):
    dep_org_prof = []
    for query in list(all_info[query_type].unique()):
        dep_org_prof.append(normalise(query))
    return dep_org_prof

In [93]:
author_name = create_list('author')
department = create_list('department') #Does not work - German
organisation = create_list('organisation')[1:] #First is nan


In [287]:
['andreas'] in author_name

True

### Query Matching - Still need to do partial Matching

In [362]:
import re
def query_match(query):
    norm_query = normalise(query)
    matching_author = [all_info['author'].unique()[i] for i,v in enumerate(author_name) if set(norm_query) == set(v)]
    #matching_org = [v for v in author_name if set(norm_query) == set(v)]
    matching_org = [v for v in organisation if set(norm_query) == set(v)]
    if matching_author == [] and matching_org == []:
         # partial_matching_authors = [sublist for sublist in author_name for item in sublist if item in norm_query]
        print('not an author')
        print('not an organisation')
    elif matching_author!=[]:
        print('--------Neo4J query: Author--------')
        print('--------Show Department--------')
        print('MATCH (p:Person) - [r:WORKS_IN] - (d)')
        print('WHERE p.name = '+'\'' +' '.join(matching_author)+'\'')
        print('Return d')
        print('--------Show Organisation--------')
        print('MATCH (p:Person) - [r:BELONGS_TO] - (d)')
        print('WHERE p.name = '+'\'' +' '.join(matching_author)+'\'')
        print('Return d')
        print('--------Show 10 most recent publications--------')
        print('MATCH (p:Person) - [r:PUBLISHED] - (d)')
        print('WHERE p.name = '+'\'' +' '.join(matching_author)+'\'')
        print('Return d')
        print('Limit 10')
        print('--------Show 10 most common collaborators--------')
        print('MATCH (p:Person)-[r1:PUBLISHED]-(pub:Publication)-[r2:PUBLISHED]-(c:Person)')
        print('WHERE p.name = '+'\'' +' '.join(matching_author)+'\'')
        print('WITH c, COUNT(pub) as cp')
        print('RETURN c, cp')
        print('ORDER BY cp DESCENDING')
        print('LIMIT 10')
        print('--------Show Expertise Areas--------')
        print('MATCH (t:Topic) - [r:EXPERT_ON] - (p:Person)')
        print('WHERE p.name = '+'\'' +' '.join(matching_author)+'\'')
        print('RETURN r.weight, t')
        print('ORDER BY r.weight DESC')
    elif matching_org != []:
        print('--------Neo4J query: Organisation--------')
        print('MATCH (d:Department) - [:WORKS_IN] - (p:Person)')
        print('WHERE p.name = ', matching_org)
        print('ORDER by d.date DESC')
        print('RETURN d')

In [1]:
def expert_on_topic(query):
    prepro_query = prepro(query)

In [363]:
query = ('Mohsen Ghaffari')#Thomas Hofmann
query_match(query)

--------Neo4J query: Author--------
--------Show Department--------
MATCH (p:Person) - [r:WORKS_IN] - (d)
WHERE p.name = 'Ghaffari, Mohsen'
Return d
--------Show Organisation--------
MATCH (p:Person) - [r:BELONGS_TO] - (d)
WHERE p.name = 'Ghaffari, Mohsen'
Return d
--------Show 10 most recent publications--------
MATCH (p:Person) - [r:PUBLISHED] - (d)
WHERE p.name = 'Ghaffari, Mohsen'
Return d
Limit 10
--------Show 10 most common collaborators--------
MATCH (p:Person)-[r1:PUBLISHED]-(pub:Publication)-[r2:PUBLISHED]-(c:Person)
WHERE p.name = 'Ghaffari, Mohsen'
WITH c, COUNT(pub) as cp
RETURN c, cp
ORDER BY cp DESCENDING
LIMIT 10


In [298]:
a = normalise(query)
a

['andreas', 'krause']

In [198]:
[sublist for sublist in author_name for item in sublist if pattern.match(item)]

[['trigo', 'isabel'],
 ['atmanspacher', 'harald'],
 ['andreasson', 'ingmar'],
 ['kabdaslı', 'isık'],
 ['diekmann', 'andreas'],
 ['sennheiser', 'andreas'],
 ['wenger', 'andreas'],
 ['horni', 'andreas'],
 ['tonnesmann', 'andreas'],
 ['schallhorn', 'andreas'],
 ['huber', 'andreas'],
 ['papritz', 'andreas', 'jurg'],
 ['bauder', 'andreas'],
 ['frei', 'andreas'],
 ['ulbig', 'andreas'],
 ['fischlin', 'andreas'],
 ['niang', 'diop', 'isabelle'],
 ['andreasen', 'lise'],
 ['hierlemann', 'andreas'],
 ['mansuy', 'isabelle'],
 ['biedermann', 'andreas'],
 ['stemmer', 'andreas'],
 ['ernst', 'andreas'],
 ['denk', 'andreas'],
 ['voigt', 'andreas'],
 ['brunschweiger', 'andreas'],
 ['mueller', 'andreas'],
 ['krause', 'andreas'],
 ['krause', 'andreas'],
 ['kispert', 'andreas'],
 ['bernasconi', 'andreas'],
 ['ruin', 'isabelle'],
 ['kunz', 'andreas'],
 ['schellenberger', 'andreas'],
 ['andreasen', 'rasmus'],
 ['isler', 'monika'],
 ['wigert', 'isabelle'],
 ['attinger'],
 ['schneider', 'andreas'],
 ['ohmura', 

In [289]:
query_match(query)

not an author
not an organisation


In [288]:
query = 'Andreas Krause is prof'

In [267]:
[sublist for sublist in author_name for item in sublist if item in norm_query]

[['diekmann', 'andreas'],
 ['sennheiser', 'andreas'],
 ['wenger', 'andreas'],
 ['horni', 'andreas'],
 ['tonnesmann', 'andreas'],
 ['schallhorn', 'andreas'],
 ['huber', 'andreas'],
 ['papritz', 'andreas', 'jurg'],
 ['bauder', 'andreas'],
 ['frei', 'andreas'],
 ['ulbig', 'andreas'],
 ['fischlin', 'andreas'],
 ['hierlemann', 'andreas'],
 ['biedermann', 'andreas'],
 ['stemmer', 'andreas'],
 ['ernst', 'andreas'],
 ['denk', 'andreas'],
 ['voigt', 'andreas'],
 ['brunschweiger', 'andreas'],
 ['mueller', 'andreas'],
 ['krause', 'andreas'],
 ['krause', 'andreas'],
 ['kispert', 'andreas'],
 ['bernasconi', 'andreas'],
 ['kunz', 'andreas'],
 ['schellenberger', 'andreas'],
 ['schneider', 'andreas'],
 ['wieck', 'andreas'],
 ['wimmer', 'andreas'],
 ['schafer', 'andreas'],
 ['pira', 'andreas'],
 ['christen', 'andreas'],
 ['kalpakci', 'andreas'],
 ['wieser', 'andreas'],
 ['schulze', 'andreas'],
 ['kuchler', 'andreas'],
 ['luscher', 'andreas'],
 ['pospischil', 'andreas'],
 ['rigling', 'andreas'],
 ['schm

In [257]:
norm_query = normalise(query)

In [101]:
matching_author = [v for v in author_name if set(query) == set(v)]

In [103]:
matching_author

[]

In [104]:
if matching_author == []:
    print('empty')

empty


### Dictionary Method

In [9]:
def create_dic(query_type):
    dep_org_prof = all_info[query_type].unique()
    dep_org_prof = dep_org_prof[1:] # removed first as nan
    dic_out = {i: True for i in dep_org_prof}
    return dic_out

In [10]:
departments = create_dic('department')
organisations = create_dic('organisation')
professor = create_dic('author')

In [78]:
departments

{'Geistes-, Sozial- und Staatswissenschaften': True,
 'Maschinenbau und Verfahrenstechnik': True,
 'Gesundheitswissenschaften und Technologie': True,
 'Architektur': True,
 'Mathematik': True,
 'Bau, Umwelt und Geomatik': True,
 'Biosysteme': True,
 'Biologie': True,
 'Umweltsystemwissenschaften': True,
 'Management, Technologie und Ökonomie': True,
 'Physik': True,
 'Chemie und Angewandte Biowissenschaften': True,
 'Informationstechnologie und Elektrotechnik': True,
 'Informatik': True,
 'Materialwissenschaft': True,
 'Erdwissenschaften': True}

In [213]:
query ='Andreas Krause'
norm_query = normalise(query)
pattern = re.compile("|".join(norm_query))
print(pattern)

re.compile('andreas|krause|is|professor|at|eth')


### Neo4J: Building the Expert score function

<br>
<br>

            #Add weights between authors and publications
            MATCH (p:Publication) - [r:PUBLISHED] - (a:Person)
            SET r.weight = 1-toFloat(duration.between(p.date, date("2020-01-01")).years)/toFloat(duration.between(date("1930-01-01"),date("2020-01-01")).years)
            
            #Add word count property to Topic nodes
            MATCH (w:Word) -[r:IS_IN] -(t:Topic)
            WITH t, count(w) as cnt
            SET t.word_count = cnt
            
            #Add publication count property to Topic nodes
            MATCH (p:Publication) -[r:IS_ABOUT] -(t:Topic)
            WITH t, count(p) as cnt
            SET t.pub_count = cnt
            
            #Create EXPERT_ON relationship between authors and topics
            #Note: To avoid memory issues, use WHERE for selection on subsets of departments
            #Ex: WHERE d.name IN ['Information Technology and Electrical Engineering', 'Mathematics']
            #Engineering choice: Exponential when computing alpha
            WITH 0.75 AS alpha_exp
            MATCH (t:Topic)-[r1:IS_ABOUT]-(p:Publication)-[r2:PUBLISHED]-(a:Person)-[r3:WORKS_IN]-(d:Department)
            WITH t, a, toFloat(count(p)) as cnt, alpha_exp
            WITH t, a,
               CASE WHEN cnt>=10.0
                  THEN 1
                  ELSE (cnt/10.0)^(alpha_exp)
               END AS alpha
            CREATE (a) - [r4:EXPERT_ON] -> (t)
            
            #Create score property on EXPERT_ON relationship
            #Again, select on department to avoid memory issues
            #Engineering choice: Exponentials for word count and publication count
            WITH 63 AS min_pub_count, 9 AS min_word_count
            MATCH (t:Topic)-[r1:IS_ABOUT]-(p:Publication)-[r2:PUBLISHED]-(a:Person)-[r3:WORKS_IN]-(d:Department)
            WITH t, a, AVG(r1.weight*r2.weight)*(min_word_count/toFloat(t.word_count))^(0.75)*(min_pub_count/toFloat(t.pub_count))^(1) as s
            MATCH (t)-[r4:EXPERT_ON]-(a)
            SET r4.score = s*r4.alpha
            
            

### Expert Query Example

<br>
<br>

            #Find experts on Climate Change
            #Find all topics that contain all search terms
            #Aggregate expert scores, by summing over all scores between topic & author
            #and weighting by the sum of the word probabilities in topic
            #Engineering choice: Sum or average to aggregate word probabilites
            WITH ['climat', 'chang']
            as words
            MATCH (w:Word)-[r1:IS_IN]-(t:Topic)
            WHERE w.name in words
            WITH t, size(words) as inputCnt, count(DISTINCT w) as cnt, SUM(r1.weight) as s
            WHERE cnt = inputCnt
            WITH  t, s
            MATCH (t:Topic)-[r3:EXPERT_ON] - (p:Person)-[r2:WORKS_IN]-(d:Department)
            WITH p, SUM(r3.score*s) as s2, d
            RETURN p.name, s2, d.name
            ORDER BY s2 DESC